## <center>Підготовка та аналіз даних</center>
## <center>Лабораторна робота №5</center>
#### <center>ФБ-25 Кравченко Максим</center>

In [2]:
import numpy as np
from IPython.display import display
import plotly.graph_objs as go
from ipywidgets import FloatSlider, Checkbox, Button, HBox, VBox

t = np.linspace(0, 1, 1000)
fs = 1000
noise = np.random.normal(0, np.sqrt(0.1), len(t))
last_noise_covariance = 0.1
initial_values = {'amplitude': 1.0, 'frequency': 1.0, 'noise_covariance': 0.1, 'show_noise': True}

def harmonic_with_noise(amplitude, frequency, noise_covariance, show_noise=True):
    global noise, last_noise_covariance, original_noisy_signal
    
    if noise_covariance != last_noise_covariance:
        noise = np.random.normal(0, np.sqrt(noise_covariance), len(t))
        last_noise_covariance = noise_covariance
    
    signal = amplitude * np.sin(2 * np.pi * frequency * t)
    original_noisy_signal = signal + noise
    
    if show_noise:
        noisy_signal = original_noisy_signal
    else:
        noisy_signal = signal
    
    return signal, noisy_signal

def custom_lowpass_filter(signal, cutoff_freq, fs=1000):
    freq_signal = np.fft.fft(signal)
    frequencies = np.fft.fftfreq(len(signal), 1/fs)
    
    cutoff = cutoff_freq * fs
    filter_mask = np.abs(frequencies) <= cutoff
    
    filtered_freq_signal = freq_signal * filter_mask
    
    filtered_signal = np.fft.ifft(filtered_freq_signal).real
    
    return filtered_signal

amplitude_slider = FloatSlider(value=initial_values['amplitude'], min=0.1, max=5.0, step=0.1, description='Amplitude:')
frequency_slider = FloatSlider(value=initial_values['frequency'], min=0.1, max=5.0, step=0.1, description='Frequency:')
noise_covariance_slider = FloatSlider(value=initial_values['noise_covariance'], min=0.01, max=1.0, step=0.01, description='Noise Covariance:')
cutoff_freq_slider = FloatSlider(value=0.1, min=0.01, max=0.5, step=0.01, description='Cutoff Frequency:')
show_noise_checkbox = Checkbox(value=True, description='Show Noisy Signal')
reset_button = Button(description='Reset', button_style='info')

signal, noisy_signal = harmonic_with_noise(**initial_values)
filtered_signal = custom_lowpass_filter(noisy_signal, cutoff_freq_slider.value)

trace_signal = go.Scatter(x=t, y=signal, mode='lines', name='Signal', line=dict(color='blue'))
trace_noisy_signal = go.Scatter(x=t, y=noisy_signal, mode='lines', name='Noisy Signal', line=dict(color='lightpink'))
trace_filtered_signal = go.Scatter(x=t, y=filtered_signal, mode='lines', name='Filtered Signal', line=dict(color='green'))

fig1 = go.FigureWidget(data=[trace_signal, trace_noisy_signal], layout=go.Layout(title='Harmonic Signal with Noise', xaxis_title='Time', yaxis_title='Value'))
fig2 = go.FigureWidget(data=[trace_filtered_signal], layout=go.Layout(title='Filtered Signal', xaxis_title='Time', yaxis_title='Value'))

def update_plot(change):
    amplitude = amplitude_slider.value
    frequency = frequency_slider.value
    noise_covariance = noise_covariance_slider.value
    show_noise = show_noise_checkbox.value
    cutoff_freq = cutoff_freq_slider.value
    
    signal, noisy_signal = harmonic_with_noise(amplitude, frequency, noise_covariance, show_noise)
    filtered_signal = custom_lowpass_filter(original_noisy_signal, cutoff_freq)
    
    with fig1.batch_update():
        fig1.data[0].y = signal
        fig1.data[1].y = noisy_signal
        fig1.data[1].visible = show_noise
        if show_noise:
            fig1.update_layout(title='Harmonic Signal with Noise')
        else:
            fig1.update_layout(title='Harmonic Signal')
    
    with fig2.batch_update():
        fig2.data[0].y = filtered_signal

def reset_values(button):
    amplitude_slider.value = initial_values['amplitude']
    frequency_slider.value = initial_values['frequency']
    noise_covariance_slider.value = initial_values['noise_covariance']
    cutoff_freq_slider.value = 0.1
    show_noise_checkbox.value = initial_values['show_noise']

reset_button.on_click(reset_values)

amplitude_slider.observe(update_plot, 'value')
frequency_slider.observe(update_plot, 'value')
noise_covariance_slider.observe(update_plot, 'value')
cutoff_freq_slider.observe(update_plot, 'value')
show_noise_checkbox.observe(update_plot, 'value')

reset_button.style.button_color = 'lightblue'
reset_button.layout.margin_left = '20px'

display(VBox([HBox([amplitude_slider, frequency_slider, noise_covariance_slider, show_noise_checkbox, reset_button]), cutoff_freq_slider, fig1, fig2]))
